In [ ]:
import requests, json
import re
import pandas as pd
from rightmove_webscraper import RightmoveData

In [ ]:
path = 'city_names.txt'
cities = []

In [ ]:
with open(path) as file:
    for line in file:
        cities.append(line.replace("\n", ""))

In [ ]:
cities = sorted(cities)
cities

In [ ]:
city_encodings = {}
path = 'https://www.rightmove.co.uk/typeAhead/uknostreet/'

In [ ]:
# Generate Rightmove the path of results page for every city
for city in cities:
    count = 0
    for char in city:
        if count == 2:
            path += '/'
            count = 0
        if char == "-" or char == " " or char == ",":
            path += '%20'
        else:
            path += char.upper()
        count += 1

    print(path)
    res = requests.get(path).json()

    # Search results dict, k = Search term, v = location ID. Ex. {'BASILDON ESSEX': '114', ...}
    locDict = {k['normalisedSearchTerm'] : k['locationIdentifier'].replace("REGION^", "") for k in res['typeAheadLocations'] if "STATION" not in k['locationIdentifier']}

    encoding = ""
    cv1 = "ZZZ"
    cv2 = "ZZZ"
    cv3 = "ZZZ"

    # Handle city names with hyphens/spaces
    if " " in city:
        cv1 = city.replace(" ", "")
    elif "-" in city:
        cv2 = city.replace("-", "")
        cv3 = city.replace("-", " ")

    # Get city encodings from locDict if city name is found
    print("City: " + city.upper())
    for key in locDict.keys():
        if city.upper() in key or cv1.upper() in key or cv2.upper() in key or cv3.upper() in key:
            encoding = locDict[key]
            break

    city_encodings[city] = encoding
    path = 'https://www.rightmove.co.uk/typeAhead/uknostreet/'


In [ ]:
path = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1271&sortType=6&propertyTypes=detached%2Csemi-detached%2Cterraced&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

In [ ]:
city_encodings, len(city_encodings)

In [ ]:
city_encodings["Bury"] = '257' # Bury and Bury St. Edmunds conflict
city_encodings["Stratford"] = '85312' # Stratford and Stratford-Upon-Avon conflict
freq = {}
for key, value in city_encodings.items():
    freq.setdefault(value, set()).add(key)

In [ ]:
# Check if there are any cities with the same encoding
[(value, key) for key, value in freq.items() if len(value) > 1]

In [ ]:
import requests
import re
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0",
    "Accept": "*/*",
    "Accept-Language": "en-GB,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Referer": "https://www.rightmove.co.uk/"
}


# Search rightmove's streetview page to return the latitude and longitude of a listing
def getLatLong(url: str, s):
    index = url.index("?")
    lat_long_url = url[:index] + "streetView"  + url[index:]
    r = requests.get(lat_long_url)
    print(lat_long_url)
    streetView_data = r.text

    regex = r'"latitude":([-]?\d{1,2}\.?\d{0,6})'
    out = re.findall(regex, streetView_data)

    if out:
        lat = out[0]
        regex = r'"longitude":([-]?\d{1,2}\.?\d{0,6})'
        out = re.findall(regex, streetView_data)
        long = out[0]
    else:
        return None

    return lat, long

In [ ]:
data = []
counts = {}
s = requests.Session()

# Scrape new house listings for each city
for city in cities:
    encoding = city_encodings[city]
    old = path[path.index("REGION%"): path.index("&")]
    new = path.replace(old, "REGION^" + encoding)

    rm = RightmoveData(new)
    df = rm.get_results

    # Add city column to df
    df['city'] = city
    df["lat"] = ""
    df["long"] = ""
    urls = df["url"]

    for i, url in enumerate(urls):
        response = getLatLong(url, s)
        if response is not None:
            lat = response[0]
            long = response[1]
            df.at[i, 'lat'] = lat
            df.at[i, 'long'] = long

    df.to_csv("data.csv", mode="a",encoding='utf-8', index=False)
    data.append(df)
    counts[city] = len(df)
    print(city, " ", counts[city], "\n\n")

In [ ]:
import pandas as pd
table = pd.read_csv("data.csv")
table.drop_duplicates(inplace=True)
table.dropna(subset=["price", "type"], inplace=True)
table.reset_index(drop=True, inplace=True)

In [43]:
table

,price,type,address,url,agent_url,postcode,full_postcode,number_bedrooms,search_date,city,lat,long
0,260000,4 bedroom semi-detached house for sale,"Johns Lane, Hirwaun, CF44 9TQ",http://www.rightmove.co.uk/properties/12539948...,http://www.rightmove.co.uk/estate-agents/agent...,CF44,CF44 9TQ,4.0,2022-09-17 20:23:35.136447,Aberdare,51.738987,-3.504362
1,145000,4 bedroom end of terrace house for sale,"Bell Street, Aberdare, Mid Glamorgan",http://www.rightmove.co.uk/properties/12717744...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,4.0,2022-09-17 20:23:35.136447,Aberdare,51.721033,-3.454723
2,170000,3 bedroom terraced house for sale,"Bute Terrace, Aberdare, Mid Glamorgan",http://www.rightmove.co.uk/properties/12539620...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,3.0,2022-09-17 20:23:35.136447,Aberdare,51.710768,-3.441247
3,175000,3 bedroom terraced house for sale,"Trevor Street, Aberdare, Mid Glamorgan",http://www.rightmove.co.uk/properties/12404807...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,3.0,2022-09-17 20:23:35.136447,Aberdare,51.709336,-3.44791
4,229995,3 bedroom semi-detached house for sale,"Derwent Drive, Aberdare, South Glamorgan, Rhon...",http://www.rightmove.co.uk/properties/12694738...,http://www.rightmove.co.uk/estate-agents/agent...,CF44,NaN,3.0,2022-09-17 20:23:35.136447,Aberdare,51.71133,-3.41749
...,...,...,...,...,...,...,...,...,...,...,...,...
86595,595000,4 bedroom end of terrace house for sale,"Charlton Street, York, YO23",http://www.rightmove.co.uk/properties/11608734...,http://www.rightmove.co.uk/estate-agents/agent...,YO23,NaN,4.0,2022-09-28 13:35:19.429534,York,53.950639,-1.084873
86596,365000,5 bedroom semi-detached house for sale,"Cherry Wood Crescent, Fulford, York",http://www.rightmove.co.uk/properties/85711197...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,5.0,2022-09-28 13:35:19.429534,York,53.92686,-1.065574
86597,1500000,5 bedroom town house for sale,"St. Martins Lane, York, YO1 6LN",http://www.rightmove.co.uk/properties/11392433...,http://www.rightmove.co.uk/estate-agents/agent...,YO1,YO1 6LN,5.0,2022-09-28 13:35:19.429534,York,53.956947,-1.08736
86598,1575000,10 bedroom semi-detached house for sale,"St. George's Place, York, YO24",http://www.rightmove.co.uk/properties/81442933...,http://www.rightmove.co.uk/estate-agents/agent...,YO24,NaN,10.0,2022-09-28 13:35:19.429534,York,53.948883,-1.101566
